In [ ]:
# I like to set up my matplotlib
# parameters so my plots look nice
# without much effort
params = {"figure.figsize": (12,9),
          "font.size": 20,
          "font.weight": "normal",
          "xtick.major.size": 9,
          "xtick.minor.size": 4,
          "ytick.major.size": 9,
          "ytick.minor.size": 4,
          "xtick.major.width": 4,
          "xtick.minor.width": 3,
          "ytick.major.width": 4,
          "ytick.minor.width": 3,
          "xtick.major.pad": 8,
          "xtick.minor.pad": 8,
          "ytick.major.pad": 8,
          "ytick.minor.pad": 8,
          "lines.linewidth": 3,
          "lines.markersize": 10,
          "axes.linewidth": 4,
          "legend.loc": "best",
          "text.usetex": False,    
          "xtick.labelsize" : 20,
          "ytick.labelsize" : 20,
          }

import matplotlib
matplotlib.rcParams.update(params)
import matplotlib.pyplot as plt

from astropy.time import Time

import pandas as pd
import glob
import numpy as np

In [ ]:
# Get a list of the light curve (LC) file names
# (Make sure you change the path here to your data files
# and that you've unzipped the folder)
lc_files = glob.glob('Basic_Lightcurves_0_zipped/*.csv')
print(F'Number of light curves in the folder: {len(lc_files)}')

In [ ]:
# Read in an example LC file
example_lc = pd.read_csv(lc_files[1], index_col=0)
# Have a look at the data
example_lc

In [ ]:
# Check the LC file column names
example_lc.columns

In [ ]:
# Convert the time column into MJD (makes everything easier to plot)
example_lc['dateobs_mjd'] = Time(np.array(example_lc['dateobs']).astype(str), format='iso').mjd

In [ ]:
# Plot the LC
fig, ax = plt.subplots(1, 1, figsize=(13, 6))
ax.errorbar(example_lc['dateobs_mjd'], example_lc['flux_peak'],
            yerr=example_lc['flux_peak_err'], fmt='o', c='Black')

# Plot the mean of the LC
xmin, xmax = ax.get_xlim()
ax.plot(np.linspace(xmin, xmax, 10), np.ones(10) * np.mean(example_lc['flux_peak']),
        '--', c='Grey', zorder=0)
ax.set_xlim(xmin, xmax)

# Add axis labels
ax.set_xlabel('Time (MJD)')
ax.set_ylabel('Flux density (mJy)')

# Add fancy labels on top with the date
axb = ax.twiny()
axb.set_xlim(ax.get_xlim())

bloc = axb.get_xticks()
bloc_mjd = Time(bloc, format='mjd')
bloc_isot = Time(bloc_mjd.isot, out_subfmt='date').value
axb.set_xticks(bloc[1:-1], bloc_isot[1:-1])
axb.xaxis.set_tick_params(labelsize=13)

# Let's calculate the variability parameters

In [ ]:
# Set up a function to calculcate the eta parameter
def eta_param(mjds, flux_densities, flux_density_errors):
    if len(mjds) > 1:
        eta = ((1 / (len(mjds) - 1)) * 
               np.sum((flux_densities - np.mean(flux_densities)) ** 2.0 /
                      flux_density_errors))
    else:
        eta = np.nan
    return eta

In [ ]:
example_eta = eta_param(example_lc['dateobs_mjd'], example_lc['flux_peak'], example_lc['flux_peak_err'])
print(example_eta)

In [ ]:
# Set up a function to calculate the V parameter
def V_param(flux_densities):
    return np.std(flux_densities) / np.mean(flux_densities)

In [ ]:
example_V = V_param(example_lc['flux_peak'])
print(example_V)

## And now we calculate the parameters for all of the sources

(This will take a minute to run, but if you have python skillz you might be able to make it faster)

In [ ]:
# Now calculate the eta and V for everything
etas = []
Vs = []

for lc_file in lc_files:
    lc_data = pd.read_csv(lc_file, index_col=0)
    lc_data['dateobs_mjd'] = Time(np.array(lc_data['dateobs']).astype(str), format='iso').mjd
    etas.append(eta_param(lc_data['dateobs_mjd'], lc_data['flux_peak'], lc_data['flux_peak_err']))
    Vs.append(V_param(lc_data['flux_peak']))

In [ ]:
# Plot the eta and V parameters
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.scatter(etas, Vs,
           marker='.', c='Black')

# Add axis labels
ax.set_xlabel(r'log$_{10}\eta$')
ax.set_ylabel(r'log$_{10}$V')

ax.set_xscale('log')
ax.set_yscale('log')

Woo! We did it! We now have the variability parameters!

But we're not done yet. Maybe you've noticed some strange things going on in this eta-V plot. Compare your eta-V plot to the ones found in the literature, for example:
- https://ui.adsabs.harvard.edu/abs/2022MNRAS.512.5037D/abstract
- https://ui.adsabs.harvard.edu/abs/2021ApJ...923...31S/abstract
- https://ui.adsabs.harvard.edu/abs/2023MNRAS.526.1888C/graphics
- https://ui.adsabs.harvard.edu/abs/2023MNRAS.523.2219A/graphics
- https://ui.adsabs.harvard.edu/abs/2023MNRAS.523.5661W/abstract

# Filter the sources

Maybe we need to filter our sources a bit before we make the eta-V plot. Things to think about include:
- variables and transients should be point sources (not resolved)
- signal-to-noise
- number of detections
- whether the source is on it's own (if it isn't it might be an artefact or part of an extended structure)

In [ ]:
# Let's try this again but with some filtering
etas_filtered = []
Vs_filtered = []
filenames = []

for lc_file in lc_files:
    lc_data = pd.read_csv(lc_file, index_col=0)

    # We only want point sources
    max_compactness = np.max(lc_data['compactness'])
    # We only want something that has at least
    # one high-ish S/N detection
    max_snr = np.max(lc_data['snr'])
    # We only want things that have a few detections
    num_detections = len(lc_data.index)
    # We only want point sources that are on their own
    has_siblings = any(lc_data['has_siblings'])

    # Set those criteria to get the "good" sources only
    if ((max_compactness < 1.5) and (max_snr > 10) and (num_detections > 5) and not has_siblings):
        lc_data['dateobs_mjd'] = Time(np.array(lc_data['dateobs']).astype(str), format='iso').mjd
        try:
            etas_filtered.append(eta_param(lc_data['dateobs_mjd'], lc_data['flux_peak'], lc_data['flux_peak_err']))
        except ZeroDivisionError:
            etas_filtered.append(np.nan)
        Vs_filtered.append(V_param(lc_data['flux_peak']))

        # It's a good idea to record which source is
        # which, so you can identify the interesting
        # ones later 
        filenames.append(lc_file)

# Store everything in a data frame, I like to do this to
# keep track of everything
var_params_filtered = pd.DataFrame(data={'filename': filenames, 'V': Vs_filtered, 'eta': etas_filtered})

In [ ]:
# Plot the eta and V parameters
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.scatter(var_params_filtered['eta'], var_params_filtered['V'],
           marker='.', c='Black')

# Add axis labels
ax.set_xlabel(r'log$_{10}\eta$')
ax.set_ylabel(r'log$_{10}$V')

ax.set_xscale('log')
ax.set_yscale('log')

# Select the interesting variable sources

Now that we have an eta-V plot, how do we use it to select variable sources?

In [ ]:
# Plot the eta and V parameters
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.scatter(np.log10(var_params['eta']), np.log10(var_params['V']),
           marker='.', c='Black')

# Add axis labels
ax.set_xlabel(r'log$_{10}\eta$')
ax.set_ylabel(r'log$_{10}$V')

xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()

# Plot the mean eta
eta_mean = np.mean(np.log10(var_params['eta']))
ax.plot(np.ones(10) * eta_mean,
        np.linspace(ymin, ymax, 10), '--', c='Gray')
# Plot the mean + 2 standard deviation of the eta
eta_mean_std = (np.mean(np.log10(var_params['eta']) + 2 * np.std(np.log10(var_params['eta']))))
ax.plot(np.ones(10) * eta_mean_std,
        np.linspace(ymin, ymax, 10), ':', c='Gray')

# Plot the mean V
V_mean = np.mean(np.log10(var_params['V']))
ax.plot(np.linspace(xmin, xmax, 10),
        np.ones(10) * V_mean,
        '--', c='Gray')
# Plot the mean + 2 standard deviation of the V
V_mean_std = (np.mean(np.log10(var_params['V']) + 2 * np.std(np.log10(var_params['V']))))
ax.plot(np.linspace(xmin, xmax, 10), 
        np.ones(10) * V_mean_std,
        ':', c='Gray')

# Shade that region
x1 = np.linspace(eta_mean_std, xmax, 10)
y1 = np.ones(10) * V_mean_std
y2 = np.ones(10) * ymax
ax.fill_between(x1, y1, y2, where=(y2 > y1), color='#DC136C', alpha=0.4)

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# For some extra plotting skills, show the histograms of the eta and V parameters as well

In [ ]:
# Select those interesting sources in the
# pandas dataframe
interesting_sources = (var_params.query(F'eta > {10.0 ** eta_mean_std} & V > {10.0 ** V_mean_std}')).sort_values(by=['eta', 'V'],
                                                                                                 ascending=False)

In [ ]:
interesting_sources

In [ ]:
is_1 = interesting_sources[interesting_sources['filename'] == 'Basic_Lightcurves_zipped/53605718.csv']
print(is_1[['eta', 'V']])

In [ ]:
is_lc = pd.read_csv(is_1['filename'].item(), index_col=0)

is_lc['dateobs_mjd'] = Time(np.array(is_lc['dateobs']).astype(str), format='iso').mjd

# Plot the LC
fig, ax = plt.subplots(1, 1, figsize=(13, 6))
ax.errorbar(is_lc['dateobs_mjd'], is_lc['flux_peak'],
            yerr=is_lc['flux_peak_err'], fmt='o', c='Black')

# Plot the mean of the LC
xmin, xmax = ax.get_xlim()
ax.plot(np.linspace(xmin, xmax, 10), np.ones(10) * np.mean(is_lc['flux_peak']),
        '--', c='Grey', zorder=0)
ax.set_xlim(xmin, xmax)

# Add axis labels
ax.set_xlabel('Time (MJD)')
ax.set_ylabel('Flux density (mJy)')

# Add fancy labels on top with the date
axb = ax.twiny()
axb.set_xlim(ax.get_xlim())

bloc = axb.get_xticks()
bloc_mjd = Time(bloc, format='mjd')
bloc_isot = Time(bloc_mjd.isot, out_subfmt='date').value
axb.set_xticks(bloc[1:-1], bloc_isot[1:-1])
axb.xaxis.set_tick_params(labelsize=13)

In [ ]:
is_lc

# Investigate some interesting sources

Use the RA, Dec and dateobs to investigate interesting variables using:
- http://cdsportal.u-strasbg.fr/
- http://simbad.u-strasbg.fr/simbad/
- https://vizier.cds.unistra.fr/viz-bin/VizieR
Or your other favourite catalogues. You might need to take proper motion into account.

If you think your source is interesting, it's a good idea to see what it looks like in the ASKAP images. Use your CASDA skills from this week to make cutouts around interesting sources to check that they aren't extended/artefacts.

Extension: polarisation properties can give big clues about what your source might be. Use CASDA to see if your source is detected in Stokes V (or Q or U).

## Everyone should investigate at least one interesting source and add their findings as one slide in the slide deck here:
https://docs.google.com/presentation/d/1eJO_RTotf3TWiUyK0jRZtQfPPxkHsXfnsu1Hy-leFaM/edit?usp=sharing

(But investigate as many as you'd like, maybe you'll find something cool!)

# Are the variability parameters working?

Are eta-V working well? Do they pick up all the possible different kinds of variability? 

Can you think of ways to test these parameters?

We're particularly interested in sources that have a single, bright flare. Do eta and V pick those up?

Are there other variability parameters that we should test instead? Maybe parameters that other wavelengths use?

# Investigating periodicity

If one of your variables looks like it might be periodic, try using the Lomb-Scargle algorithm to find the period: https://astropy-cjhang.readthedocs.io/en/latest/stats/lombscargle.html 

There's also a Scipy Lomb-Scargle, do you get the same results for both?

# Want a bigger sample size?

Add in the data from the other folders.